In [1]:
import os
os.chdir("../")

In [2]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [3]:
from src.MatchAnalysis.constants import *
from src.MatchAnalysis.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self) -> DataIngestionConfig:

        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir        = config.root_dir,
            source_URL      = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir       = config.unzip_dir
        )

        return data_ingestion_config

In [5]:
import os
import zipfile
import gdown
from src.MatchAnalysis import logger
from src.MatchAnalysis.utils.common import get_size

/Users/guido/Documents/match-analysis/.venv/lib/python3.9/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
class DatatIngestion:

    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self) -> str:
        """ 
        Fetch data from the url
        """

        try:
            dataset_url = self.config.source_URL
            csv_download_dir = self.config.local_data_file
            os.makedirs(self.config.root_dir, exist_ok=True)
            logger.info(f'Downloading data from {dataset_url} to {csv_download_dir}')

            file_id = dataset_url.split('/')[-2]
            prefix = 'https://drive.google.com/uc?/export-download&id='
            gdown.download(prefix + file_id, csv_download_dir)

            logger.info(f'downloaded data from {dataset_url} to {csv_download_dir}')

        except Exception as e:
            raise e

    def extract_zip_file(self):
        """
        zip_file_path: str
        extracts the zip file into the data directory
        function returns None
        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [7]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DatatIngestion(data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-01-26 11:03:24,509: INFO: common] yaml file: config/config.yaml loaded successfully
[2024-01-26 11:03:24,511: INFO: common] yaml file: params.yaml loaded successfully
[2024-01-26 11:03:24,511: INFO: common] Creating directory: artifacts
[2024-01-26 11:03:24,512: INFO: common] Creating directory: artifacts/data_ingestion
[2024-01-26 11:03:24,512: INFO: 65794666] Downloading data from https://drive.google.com/file/d/1YbawRTUPhkdgdqUUJ9BJuI1Z3SSRoWs3/view?usp=sharing to artifacts/data_ingestion/data.zip


Downloading...
From: https://drive.google.com/uc?/export-download&id=1YbawRTUPhkdgdqUUJ9BJuI1Z3SSRoWs3
To: /Users/guido/Documents/match-analysis/artifacts/data_ingestion/data.zip
100%|██████████| 1.40M/1.40M [00:00<00:00, 5.30MB/s]

[2024-01-26 11:03:26,287: INFO: 65794666] downloaded data from https://drive.google.com/file/d/1YbawRTUPhkdgdqUUJ9BJuI1Z3SSRoWs3/view?usp=sharing to artifacts/data_ingestion/data.zip
